In [1]:
import os
import re
import sys
import pandas as pd
sys.path.insert(0, './_modules')
from db_functions import TheDB
#from logging_module import getlogger
from logging_module_v2 import StartLogger
from regex_module import FindEmail, FindNumbersBraket, FindLawsuit, FindCaseId
import ast
from sqlalchemy import create_engine
import pymysql
import random

### Data

In [2]:
adv_list = ['Carina Quito',
'Rafael Nogaroto Kohl',
'Vitor Honofre Bellotto',
'Alice Christina Matsuo',
'Helena Costa Rossi',
'Mohamad Hassan Fares',
'Artur Rovere']

In [3]:
firstname_list = [item.split()[0] for item in adv_list]
firstname_list

['Carina', 'Rafael', 'Vitor', 'Alice', 'Helena', 'Mohamad', 'Artur']

### Functions

In [4]:
def MailKiller(input_string):
    final_string = re.sub('<.*?>','', input_string, flags=re.DOTALL)
    return final_string.strip()

In [5]:
def MatchNames(input_list):
    return [item for item in input_list if any(name in item for name in firstname_list)]

In [6]:
def GetFirstName(input_string):
    return input_string.split()[0]


def GetLastName(input_string):
    return input_string.split()[-1]

In [7]:
def GetFirst(input_list, default = ''):
    
    try:
        result = input_list[0]
    except:
        result = default
    
    return result

### BD connection

In [8]:
## bots BD connections
conn_dict = {'host':'localhost', 'database': 'brspcqadev',
             'user':'root', 'password':'quito2020'}

cqdb = TheDB(conn_dict)

### Paths and Data:

In [9]:
data_path = os.path.join(os.getcwd(), '_data_output')

In [10]:
gmail_path = os.path.join(os.getcwd(), '_data_original')

In [11]:
## modelo de tabela para imputar dados na tabela Trix
df_model = pd.read_csv('model_trix_db.csv', sep = '\t')

In [12]:
## modelo de tabela pra imputar dados na tabela Users
df_users_model = pd.read_csv('users_table_model.csv', sep = '\t', encoding = 'utf-8')

### cadastrando usuários no BD:

In [13]:
df_users = cqdb.GetTable('Users')

In [14]:
cqdb.TablesDataType('Users')

,0,1,2,3,4,5
0,users_id,int(11),NO,PRI,None,auto_increment
1,first_name,longtext,YES,,None,
2,last_name,longtext,YES,,None,
3,email,longtext,YES,,None,
4,cpf,varchar(45),YES,,None,
5,rg,longtext,YES,,None,
6,oab,longtext,YES,,None,
7,phone,longtext,YES,,None,
8,cellphone,longtext,YES,,None,
9,Password,longtext,YES,,None,


In [15]:
df_users_model

,first_name,last_name,email,cpf,rg,oab,phone,cellphone,Password,users_status_id,role_id,level_id,ip_id,adress_id
0,Ana,Paula,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,0.000000e+00,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
1,Joaquina,Trix,joaquinatrix@carinaquito.com,123.545.845-58,35.484.566-4,1.235479e+20,(11) 2 2255-4588,(85) 4 8455-5554,8CB2237D0679CA88DB6464EAC60DA96345513964,1,2,2,1,32


In [16]:
## criando df para hospedar novos users:
df_new = pd.DataFrame(columns = df_users_model.columns)
columns_list = df_new.columns
columns_list

Index(['first_name', 'last_name', 'email', 'cpf', 'rg', 'oab', 'phone',
       'cellphone', 'Password', 'users_status_id', 'role_id', 'level_id',
       'ip_id', 'adress_id'],
      dtype='object')

In [17]:
## copiando os dados que não serão alterados da tabela modelo
base_mail = df_users_model['email'][1]
base_cpf = df_users_model['cpf'][1]
base_rg = df_users_model['rg'][1]
base_oab = df_users_model['oab'][1]
base_phone = df_users_model['phone'][1]
base_cellphone = df_users_model['cellphone'][1]
base_password = df_users_model['Password'][1]
base_users_status_id = df_users_model['users_status_id'][1]
base_role_id = df_users_model['role_id'][1]
base_level_id = df_users_model['level_id'][1]
base_ip_id= df_users_model['ip_id'][1]
base_address_id = df_users_model['adress_id'][1]

In [18]:
## populando o df_new
idx = 0
for fullname in adv_list:
    
    firstname = fullname.split()[0]
    lastname = fullname.split()[-1]
    df_new.at[idx, 'first_name'] = firstname
    df_new.at[idx, 'last_name'] = lastname
    
    df_new.at[idx, 'email'] = base_mail
    df_new.at[idx, 'cpf'] = base_cpf 
    df_new.at[idx, 'rg'] = base_rg
    df_new.at[idx, 'oab'] = base_oab
    df_new.at[idx, 'phone'] = base_phone
    df_new.at[idx, 'cellphone'] = base_cellphone
    df_new.at[idx, 'Password'] = base_password
    df_new.at[idx, 'users_status_id'] = base_users_status_id
    df_new.at[idx, 'role_id'] = base_role_id
    df_new.at[idx, 'level_id'] = base_level_id
    df_new.at[idx, 'ip_id'] = base_ip_id
    df_new.at[idx, 'adress_id'] = base_address_id
    
    
        
        
        
    idx += 1

In [19]:
## cadastrando no DB:
write_db = False

if write_db == True:
    cqdb.InsertDB(df_new, 'Users', key_column='first_name', return_table=False, append_table=True)

In [20]:
## checando a nova tabela:
df_users = cqdb.GetTable('Users')

In [21]:
## mantendo apenas as colunas de interesse:
df_users = df_users.loc[:, ['first_name', 'users_id']]
df_users

,first_name,users_id
0,Admin,1
1,Ana,14
2,Joaquina,15
3,Ana2,17
4,Joaquina2,18
5,Carina,19
6,Rafael,20
7,Vitor,21
8,Alice,22
9,Helena,23


### Working on email data

In [22]:
df_mail = pd.read_csv(os.path.join(gmail_path, 'emails.csv'), sep = '\t', encoding = 'utf-8')
df_mail = df_mail.drop('Unnamed: 0', axis = 1)
df_mail['line_id'] = df_mail['numbers_list'].apply(ast.literal_eval).apply(GetFirst)
df_mail['line_id'] = df_mail['line_id'].astype(int)
df_mail

,path,line_in_path,sender,line_in_mail_filename,trix_counter,numbers_list,numbers_counting,match,line_id
0,/home/leandro/Desktop/Gmail/Linha 9159/_3-8493...,Linha 9159,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9159'],1,['9159'],1,True,9159
1,/home/leandro/Desktop/Gmail/Linha 9692/Re_ RES...,Linha 9692,lis-latam@google.com,['9692'],1,['9692'],1,True,9692
2,/home/leandro/Desktop/Gmail/Linha 9692/RES_ _0...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692
3,/home/leandro/Desktop/Gmail/Linha 9692/_4-0944...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692
4,/home/leandro/Desktop/Gmail/Linha 9692/RES_ _0...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692
...,...,...,...,...,...,...,...,...,...
19042,/home/leandro/Desktop/Gmail/Linha 9515/_8-7268...,Linha 9515,Helena Costa Rossi,['9515'],1,['9515'],1,True,9515
19043,/home/leandro/Desktop/Gmail/Linha 10093 - Lega...,Linha 10093 - Legal Issue,Rafael Nogaroto Kohl <rafael.kohl@carinaquito....,['10093'],1,['10093'],1,True,10093
19044,/home/leandro/Desktop/Gmail/Linha 10093 - Lega...,Linha 10093 - Legal Issue,lis-latam via Google <google@carinaquito.com>,['10093'],1,['10093'],1,True,10093
19045,/home/leandro/Desktop/Gmail/Linha 9741/_4-4742...,Linha 9741,Helena Costa Rossi <helena.rossi@carinaquito.com>,['9741'],1,['9741'],1,True,9741


In [23]:
## removendo linhas com email com domínio "@google", pois não são dos advs do escritorio
df_mail = df_mail[df_mail['sender'].str.contains('@google') == False]


In [24]:
## removendo o email do campo sender e mantendo apenas o nome
df_mail['sender_clean'] = df_mail['sender'].apply(MailKiller)

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
## agrupando os remetentes por linha:
df_grouped = df_mail.groupby('line_id')['sender_clean'].apply(list).apply(set).apply(list).\
to_frame().reset_index()
df_grouped

,line_id,sender_clean
0,55,[Carina Quito]
1,149,[S & Q Advogados]
2,330,[S & Q Advogados]
3,367,[Alice Christina Matsuo]
4,391,[S & Q Advogados]
...,...,...
8331,10461,[Rafael Nogaroto Kohl]
8332,10462,[Vitor Honofre Bellotto]
8333,10466,[Rafael Nogaroto Kohl]
8334,10467,[Rafael Nogaroto Kohl]


In [26]:
## verificando quais linhas tem algum match com os nomes dos advs atuais:
df_grouped['sender_clean_match'] = df_grouped['sender_clean'].apply(MatchNames)
df_grouped

,line_id,sender_clean,sender_clean_match
0,55,[Carina Quito],[Carina Quito]
1,149,[S & Q Advogados],[]
2,330,[S & Q Advogados],[]
3,367,[Alice Christina Matsuo],[Alice Christina Matsuo]
4,391,[S & Q Advogados],[]
...,...,...,...
8331,10461,[Rafael Nogaroto Kohl],[Rafael Nogaroto Kohl]
8332,10462,[Vitor Honofre Bellotto],[Vitor Honofre Bellotto]
8333,10466,[Rafael Nogaroto Kohl],[Rafael Nogaroto Kohl]
8334,10467,[Rafael Nogaroto Kohl],[Rafael Nogaroto Kohl]


In [27]:
## Extraindo das listas o nome completo do adv. Em casos de listas com mais de um elemento, pega-se
## o primeiro nome (pois não a critérios para saber quem seria o responsável pela linha.)
## Quando a lista pe vazia, atribui-se a linha à Carina
df_grouped['fullname'] = df_grouped['sender_clean_match'].apply(GetFirst, default = 'Carina Quito')
df_grouped

,line_id,sender_clean,sender_clean_match,fullname
0,55,[Carina Quito],[Carina Quito],Carina Quito
1,149,[S & Q Advogados],[],Carina Quito
2,330,[S & Q Advogados],[],Carina Quito
3,367,[Alice Christina Matsuo],[Alice Christina Matsuo],Alice Christina Matsuo
4,391,[S & Q Advogados],[],Carina Quito
...,...,...,...,...
8331,10461,[Rafael Nogaroto Kohl],[Rafael Nogaroto Kohl],Rafael Nogaroto Kohl
8332,10462,[Vitor Honofre Bellotto],[Vitor Honofre Bellotto],Vitor Honofre Bellotto
8333,10466,[Rafael Nogaroto Kohl],[Rafael Nogaroto Kohl],Rafael Nogaroto Kohl
8334,10467,[Rafael Nogaroto Kohl],[Rafael Nogaroto Kohl],Rafael Nogaroto Kohl


In [28]:
## pegando o primeiro e último nome
df_grouped['first_name'] = df_grouped['fullname'].apply(GetFirstName)
df_grouped['last_name'] = df_grouped['fullname'].apply(GetLastName)
df_grouped

,line_id,sender_clean,sender_clean_match,fullname,first_name,last_name
0,55,[Carina Quito],[Carina Quito],Carina Quito,Carina,Quito
1,149,[S & Q Advogados],[],Carina Quito,Carina,Quito
2,330,[S & Q Advogados],[],Carina Quito,Carina,Quito
3,367,[Alice Christina Matsuo],[Alice Christina Matsuo],Alice Christina Matsuo,Alice,Matsuo
4,391,[S & Q Advogados],[],Carina Quito,Carina,Quito
...,...,...,...,...,...,...
8331,10461,[Rafael Nogaroto Kohl],[Rafael Nogaroto Kohl],Rafael Nogaroto Kohl,Rafael,Kohl
8332,10462,[Vitor Honofre Bellotto],[Vitor Honofre Bellotto],Vitor Honofre Bellotto,Vitor,Bellotto
8333,10466,[Rafael Nogaroto Kohl],[Rafael Nogaroto Kohl],Rafael Nogaroto Kohl,Rafael,Kohl
8334,10467,[Rafael Nogaroto Kohl],[Rafael Nogaroto Kohl],Rafael Nogaroto Kohl,Rafael,Kohl


In [29]:
df_grouped['first_name'].value_counts()

Rafael      1875
Carina      1608
Mohamad     1034
"Helena      992
Helena       941
Vitor        599
"Rafael      556
"Mohamad     461
"Alice       128
Alice         59
"Vitor        51
"Carina       27
Artur          5
Name: first_name, dtype: int64

In [30]:
## limpando os nomes
df_grouped['first_name'] = df_grouped['first_name'].str.replace('"', '')

In [31]:
## salvando para arquivo
df_grouped.loc[:, ['line_id', 'first_name', 'last_name',\
                   'fullname']].to_csv('legacy_name_trix_id.csv',sep = '\t', encoding = 'utf-8', index = False)

In [32]:
## ficando com o que é importante:
df_grouped2 = df_grouped.loc[:, ['line_id', 'first_name']]
df_grouped2

,line_id,first_name
0,55,Carina
1,149,Carina
2,330,Carina
3,367,Alice
4,391,Carina
...,...,...
8331,10461,Rafael
8332,10462,Vitor
8333,10466,Rafael
8334,10467,Rafael


### Manipulating the Trix file

In [33]:
## carregando o arquivo e selecionando as colunas que serão passíveis de subir:
df = pd.read_csv(os.path.join(data_path, 'main_trix_clean_v2.csv'), sep = '\t')
print(df.columns)
df = df.rename(columns = {'Case # (DLBCA and Google reference)':'case_id',\
                     'line':'line_id', 'Lawsuit Number':'external_ref_number', 'Deadline':'deadline'})
df = df.loc[:, ['line_id', 'case_id', 'external_ref_number', 'deadline']]
df['external_ref_number'] = df['external_ref_number'].apply(FindLawsuit)
df['case_id'] = df['case_id'].apply(FindCaseId)

Index(['line', 'Case # (DLBCA and Google reference)', 'UID',
       'Date of the Court Orders', 'Court', 'same court', 'Lawsuit Number',
       'CP?', 'Summary of the request/what to produce [internal team rec]',
       'STQ confirm scope of production', 'Deadline',
       'DLBCA's suggested answer', 'Previous Orders and Answers',
       'Google's proposed answer', 'LIS: \nInformatin about case',
       'LIS: \nPolicy and content check',
       'LIS: \nGoogle Inc answer sent to authority? which one?',
       'LIS:\nPriority', 'LIS:\nAssignee', 'Status',
       'Status - writ of mandamus', 'Law Firm'],
      dtype='object')


In [34]:
df

,line_id,case_id,external_ref_number,deadline
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00
3,4,5-6191000002205,,2013-11-22 00:00:00
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00
...,...,...,...,...
10305,10542,6-2387000030981,0000880-18.2020.8.03.0006,2020-09-04 00:00:00
10306,10546,8-1172000030681,5005563-26.2020.4.02.5001,2020-08-31 00:00:00
10307,10548,4-9100000031065,1042340-66.2019.8.13.0024,2020-08-24 00:00:00
10308,10549,3-5849000026727,014055-55.2019.8.06.0001,2020-08-31 00:00:00


In [35]:
## input some date, since those fields are manadatory
df['date_received'] = '1946-02-12'
df['date_received_cq'] = '1946-02-12'

In [36]:
## ficando com as linhas cujo line_id é número
df = df[df['line_id'].str.isdigit() == True]
df['line_id'] = df['line_id'].astype(int)

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
df

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,1946-02-12,1946-02-12
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,1946-02-12,1946-02-12
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,1946-02-12,1946-02-12
3,4,5-6191000002205,,2013-11-22 00:00:00,1946-02-12,1946-02-12
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,1946-02-12,1946-02-12
...,...,...,...,...,...,...
10305,10542,6-2387000030981,0000880-18.2020.8.03.0006,2020-09-04 00:00:00,1946-02-12,1946-02-12
10306,10546,8-1172000030681,5005563-26.2020.4.02.5001,2020-08-31 00:00:00,1946-02-12,1946-02-12
10307,10548,4-9100000031065,1042340-66.2019.8.13.0024,2020-08-24 00:00:00,1946-02-12,1946-02-12
10308,10549,3-5849000026727,014055-55.2019.8.06.0001,2020-08-31 00:00:00,1946-02-12,1946-02-12


In [38]:
df = df.drop_duplicates(subset = ['line_id'])

In [39]:
df['date_received'] = pd.to_datetime(df['date_received'])
df['date_received_cq'] = pd.to_datetime(df['date_received_cq'])

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
## capturando as colunas do df modelo
columns_list = list(df_model.columns)

In [41]:
## criando o novo df para abrigar as linhas legadas:
df_clean = pd.DataFrame(columns = columns_list)

In [42]:
## compondo o df final da Trix legada
df_final = pd.concat([df_clean, df], sort = False).loc[:, columns_list].fillna('')
df_final

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,1946-02-12,1946-02-12,,,,,...,,,,,,,,,,
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,1946-02-12,1946-02-12,,,,,...,,,,,,,,,,
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,1946-02-12,1946-02-12,,,,,...,,,,,,,,,,
3,4,5-6191000002205,,2013-11-22 00:00:00,1946-02-12,1946-02-12,,,,,...,,,,,,,,,,
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,1946-02-12,1946-02-12,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10305,10542,6-2387000030981,0000880-18.2020.8.03.0006,2020-09-04 00:00:00,1946-02-12,1946-02-12,,,,,...,,,,,,,,,,
10306,10546,8-1172000030681,5005563-26.2020.4.02.5001,2020-08-31 00:00:00,1946-02-12,1946-02-12,,,,,...,,,,,,,,,,
10307,10548,4-9100000031065,1042340-66.2019.8.13.0024,2020-08-24 00:00:00,1946-02-12,1946-02-12,,,,,...,,,,,,,,,,
10308,10549,3-5849000026727,014055-55.2019.8.06.0001,2020-08-31 00:00:00,1946-02-12,1946-02-12,,,,,...,,,,,,,,,,


In [43]:
## preenchendo os dados not null com dados fake:
df_final['status'] = 'migração'
df_final['label'] = df_model['label'][0]
df_final['date_created'] = df_model['date_created'][0]
df_final['date_modified'] = df_model['date_modified'][0]
df_final['users_id'] = df_model['users_id'][0]
df_final['user_advice_correction'] = df_model['user_advice_correction'][0]
df_final['advice_approved'] = df_model['advice_approved'][0]
df_final['advice_sent'] = df_model['advice_sent'][0]
df_final['petition_cat_id'] = df_model['petition_cat_id'][0]
df_final['date_status_ms'] = df_model['date_status_ms'][0]
df_final['deadline_ms'] = df_model['deadline_ms'][0]
df_final['user_in_charge_production'] = df_model['user_in_charge_production'][0]
df_final['monitor'] = df_model['monitor'][0]
df_final['emissary_chosen'] = df_model['emissary_chosen'][0]



## fillna
df_final['deadline'].fillna('2020-08-20', inplace = True)

In [44]:
df_final

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,1946-02-12,1946-02-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,1946-02-12,1946-02-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,1946-02-12,1946-02-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
3,4,5-6191000002205,,2013-11-22 00:00:00,1946-02-12,1946-02-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,1946-02-12,1946-02-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10305,10542,6-2387000030981,0000880-18.2020.8.03.0006,2020-09-04 00:00:00,1946-02-12,1946-02-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
10306,10546,8-1172000030681,5005563-26.2020.4.02.5001,2020-08-31 00:00:00,1946-02-12,1946-02-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
10307,10548,4-9100000031065,1042340-66.2019.8.13.0024,2020-08-24 00:00:00,1946-02-12,1946-02-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
10308,10549,3-5849000026727,014055-55.2019.8.06.0001,2020-08-31 00:00:00,1946-02-12,1946-02-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,


In [45]:
df_final['deadline'] = df_final['deadline'].replace('', '2020-08-20')
df_final['deadline'] = pd.to_datetime(df_final['deadline'])

In [46]:
df_final = df_final.drop('line_id', axis = 1)

In [47]:
df_final

,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,status,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
0,,0009460-19.2013.403.6181,2014-02-28,1946-02-12,1946-02-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
1,6-9080000028545,0002618-91.2011.403.6181,2013-11-28,1946-02-12,1946-02-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
2,9-5821000001559,0007067-24.2013.403.6181,2013-10-22,1946-02-12,1946-02-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
3,5-6191000002205,,2013-11-22,1946-02-12,1946-02-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
4,9-9709000000828,0076690-72.2011.805.0001,2013-10-21,1946-02-12,1946-02-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10305,6-2387000030981,0000880-18.2020.8.03.0006,2020-09-04,1946-02-12,1946-02-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
10306,8-1172000030681,5005563-26.2020.4.02.5001,2020-08-31,1946-02-12,1946-02-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
10307,4-9100000031065,1042340-66.2019.8.13.0024,2020-08-24,1946-02-12,1946-02-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
10308,3-5849000026727,014055-55.2019.8.06.0001,2020-08-31,1946-02-12,1946-02-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,


In [48]:
df_final.columns

Index(['case_id', 'external_ref_number', 'deadline', 'date_received',
       'date_received_cq', 'auth_last_name', 'info_case', 'case_country',
       'email_response', 'status', 'targets', 'lers', 'label', 'date_created',
       'date_modified', 'users_id', 'legal_opinion_background',
       'legal_opinion_legal_risk', 'legal_opinion_additional_info',
       'legal_opinion_issue_comments', 'legal_opinion_silo',
       'legal_opinion_petition_comments', 'user_advice_correction',
       'council_review_letter', 'writ_filed_date', 'daily_fines',
       'freezing_assets_date', 'advice_approved', 'advice_pending_yes_no',
       'advice_pendency', 'advice_pendency_solved', 'advice_sent',
       'petition_cat_id', 'petition_yes_no_pending', 'petition_no_explanation',
       'petition_body', 'petition_draft_prepared', 'petition_draft_corrected',
       'petition_field', 'ms_draft_write', 'ms_draft_reviewed', 'ms_filed',
       'status_ms', 'date_status_ms', 'deadline_ms', 'ms_info',
       'l

### The final merge

In [49]:
df_legacy = df_users.loc[:, ['first_name', 'users_id']].merge(df_grouped2, on = 'first_name', how = 'right')
df_legacy

,first_name,users_id,line_id
0,Carina,19,55
1,Carina,19,149
2,Carina,19,330
3,Carina,19,391
4,Carina,19,770
...,...,...,...
8331,Artur,25,9517
8332,Artur,25,9979
8333,Artur,25,10007
8334,Artur,25,10206


In [50]:
df

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,1946-02-12,1946-02-12
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,1946-02-12,1946-02-12
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,1946-02-12,1946-02-12
3,4,5-6191000002205,,2013-11-22 00:00:00,1946-02-12,1946-02-12
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,1946-02-12,1946-02-12
...,...,...,...,...,...,...
10305,10542,6-2387000030981,0000880-18.2020.8.03.0006,2020-09-04 00:00:00,1946-02-12,1946-02-12
10306,10546,8-1172000030681,5005563-26.2020.4.02.5001,2020-08-31 00:00:00,1946-02-12,1946-02-12
10307,10548,4-9100000031065,1042340-66.2019.8.13.0024,2020-08-24 00:00:00,1946-02-12,1946-02-12
10308,10549,3-5849000026727,014055-55.2019.8.06.0001,2020-08-31 00:00:00,1946-02-12,1946-02-12


In [51]:
df2 = df.merge(df_legacy, on = 'line_id', how = 'left').drop('first_name', axis = 1)
df2

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,users_id
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,1946-02-12,1946-02-12,NaN
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,1946-02-12,1946-02-12,NaN
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,1946-02-12,1946-02-12,NaN
3,4,5-6191000002205,,2013-11-22 00:00:00,1946-02-12,1946-02-12,NaN
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,1946-02-12,1946-02-12,NaN
...,...,...,...,...,...,...,...
10278,10542,6-2387000030981,0000880-18.2020.8.03.0006,2020-09-04 00:00:00,1946-02-12,1946-02-12,NaN
10279,10546,8-1172000030681,5005563-26.2020.4.02.5001,2020-08-31 00:00:00,1946-02-12,1946-02-12,NaN
10280,10548,4-9100000031065,1042340-66.2019.8.13.0024,2020-08-24 00:00:00,1946-02-12,1946-02-12,NaN
10281,10549,3-5849000026727,014055-55.2019.8.06.0001,2020-08-31 00:00:00,1946-02-12,1946-02-12,NaN


In [52]:
## algumas linhas ficaram sem users_id atribuido. 
## Isso significa que provavelmente os dados não puderam ser recuperados dos emails legados
## Decisão: atribuir ao user da carina:

In [53]:
carina_id = df_users[df_users['first_name'] == 'Carina']['users_id'].values[0]

In [54]:
df2['users_id'] = df2['users_id'].fillna(carina_id).astype(int)

In [55]:
df2

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,users_id
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,1946-02-12,1946-02-12,19
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,1946-02-12,1946-02-12,19
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,1946-02-12,1946-02-12,19
3,4,5-6191000002205,,2013-11-22 00:00:00,1946-02-12,1946-02-12,19
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,1946-02-12,1946-02-12,19
...,...,...,...,...,...,...,...
10278,10542,6-2387000030981,0000880-18.2020.8.03.0006,2020-09-04 00:00:00,1946-02-12,1946-02-12,19
10279,10546,8-1172000030681,5005563-26.2020.4.02.5001,2020-08-31 00:00:00,1946-02-12,1946-02-12,19
10280,10548,4-9100000031065,1042340-66.2019.8.13.0024,2020-08-24 00:00:00,1946-02-12,1946-02-12,19
10281,10549,3-5849000026727,014055-55.2019.8.06.0001,2020-08-31 00:00:00,1946-02-12,1946-02-12,19


In [56]:
df2

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,users_id
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,1946-02-12,1946-02-12,19
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,1946-02-12,1946-02-12,19
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,1946-02-12,1946-02-12,19
3,4,5-6191000002205,,2013-11-22 00:00:00,1946-02-12,1946-02-12,19
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,1946-02-12,1946-02-12,19
...,...,...,...,...,...,...,...
10278,10542,6-2387000030981,0000880-18.2020.8.03.0006,2020-09-04 00:00:00,1946-02-12,1946-02-12,19
10279,10546,8-1172000030681,5005563-26.2020.4.02.5001,2020-08-31 00:00:00,1946-02-12,1946-02-12,19
10280,10548,4-9100000031065,1042340-66.2019.8.13.0024,2020-08-24 00:00:00,1946-02-12,1946-02-12,19
10281,10549,3-5849000026727,014055-55.2019.8.06.0001,2020-08-31 00:00:00,1946-02-12,1946-02-12,19


### Finding the missing line_ids:

In [57]:
line_id_list = df2['line_id'].tolist()

In [58]:
def missing_elements(L):
    start, end = L[0], L[-1]
    return sorted(set(range(start, end + 1)).difference(L))

In [59]:
missing_list = missing_elements(line_id_list)
len(missing_list)

268

In [60]:
df3 = pd.DataFrame(columns = df2.columns)

In [61]:
## preenchendo as linhas vazias:
df3['line_id'] = missing_list
df3['deadline'] = random.sample(df2['date_received'].tolist(), 1)*len(missing_list)
df3['date_received'] = random.sample(df2['date_received'].tolist(), 1)*len(missing_list)
df3['date_received_cq'] = random.sample(df2['date_received_cq'].tolist(), 1)*len(missing_list)
df3['users_id'] = [19]*len(missing_list)
df3['case_id'] = ['missing_value']*len(missing_list)
df3['external_ref_number'] = ['missing_value']*len(missing_list)

In [62]:
df3

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,users_id
0,21,missing_value,missing_value,1946-02-12,1946-02-12,1946-02-12,19
1,23,missing_value,missing_value,1946-02-12,1946-02-12,1946-02-12,19
2,26,missing_value,missing_value,1946-02-12,1946-02-12,1946-02-12,19
3,28,missing_value,missing_value,1946-02-12,1946-02-12,1946-02-12,19
4,36,missing_value,missing_value,1946-02-12,1946-02-12,1946-02-12,19
...,...,...,...,...,...,...,...
263,10543,missing_value,missing_value,1946-02-12,1946-02-12,1946-02-12,19
264,10544,missing_value,missing_value,1946-02-12,1946-02-12,1946-02-12,19
265,10545,missing_value,missing_value,1946-02-12,1946-02-12,1946-02-12,19
266,10547,missing_value,missing_value,1946-02-12,1946-02-12,1946-02-12,19


In [63]:
df_final = pd.concat([df2, df3]).sort_values('line_id', ascending = True).reset_index(drop = True)
df_final

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,users_id
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,1946-02-12,1946-02-12,19
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,1946-02-12,1946-02-12,19
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,1946-02-12,1946-02-12,19
3,4,5-6191000002205,,2013-11-22 00:00:00,1946-02-12,1946-02-12,19
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,1946-02-12,1946-02-12,19
...,...,...,...,...,...,...,...
10546,10547,missing_value,missing_value,1946-02-12 00:00:00,1946-02-12,1946-02-12,19
10547,10548,4-9100000031065,1042340-66.2019.8.13.0024,2020-08-24 00:00:00,1946-02-12,1946-02-12,19
10548,10549,3-5849000026727,014055-55.2019.8.06.0001,2020-08-31 00:00:00,1946-02-12,1946-02-12,19
10549,10550,missing_value,missing_value,1946-02-12 00:00:00,1946-02-12,1946-02-12,19


In [69]:
df_final['date_created'] = df_final['date_received'].tolist() 
df_final['date_modified'] = df_final['date_received'].tolist() 
df_final['date_status_ms'] = df_final['date_received'].tolist() 
df_final['deadline_ms'] = df_final['date_received'].tolist() 
df_final['deadline'] = df_final['date_received'].tolist()

In [70]:
## to datetime
df_final['deadline'] = pd.to_datetime(df_final['deadline'])
df_final['date_received'] = pd.to_datetime(df_final['date_received'])
df_final['date_received_cq'] = pd.to_datetime(df_final['date_received_cq'])

df_final['date_modified'] = pd.to_datetime(df_final['date_modified'])
df_final['date_status_ms'] =pd.to_datetime(df_final['date_status_ms'])
df_final['deadline_ms'] =pd.to_datetime(df_final['deadline_ms'])
df_final['deadline'] = pd.to_datetime(df_final['deadline'])

In [71]:
df_final

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,users_id,date_created,date_modified,date_status_ms,deadline_ms
0,1,,0009460-19.2013.403.6181,1946-02-12,1946-02-12,1946-02-12,19,1946-02-12,1946-02-12,1946-02-12,1946-02-12
1,2,6-9080000028545,0002618-91.2011.403.6181,1946-02-12,1946-02-12,1946-02-12,19,1946-02-12,1946-02-12,1946-02-12,1946-02-12
2,3,9-5821000001559,0007067-24.2013.403.6181,1946-02-12,1946-02-12,1946-02-12,19,1946-02-12,1946-02-12,1946-02-12,1946-02-12
3,4,5-6191000002205,,1946-02-12,1946-02-12,1946-02-12,19,1946-02-12,1946-02-12,1946-02-12,1946-02-12
4,5,9-9709000000828,0076690-72.2011.805.0001,1946-02-12,1946-02-12,1946-02-12,19,1946-02-12,1946-02-12,1946-02-12,1946-02-12
...,...,...,...,...,...,...,...,...,...,...,...
10546,10547,missing_value,missing_value,1946-02-12,1946-02-12,1946-02-12,19,1946-02-12,1946-02-12,1946-02-12,1946-02-12
10547,10548,4-9100000031065,1042340-66.2019.8.13.0024,1946-02-12,1946-02-12,1946-02-12,19,1946-02-12,1946-02-12,1946-02-12,1946-02-12
10548,10549,3-5849000026727,014055-55.2019.8.06.0001,1946-02-12,1946-02-12,1946-02-12,19,1946-02-12,1946-02-12,1946-02-12,1946-02-12
10549,10550,missing_value,missing_value,1946-02-12,1946-02-12,1946-02-12,19,1946-02-12,1946-02-12,1946-02-12,1946-02-12


In [72]:
## Escrevendo no DB:
send2DB = True

if send2DB == True:


    ## create sqlalchemy engine
    engine = create_engine("mysql://{user}:{pw}@{host}/{db}?charset=utf8mb4"
                           .format(user= conn_dict['user'],
                                   pw=conn_dict['password'],
                                   host = conn_dict['host'],
                                   db=conn_dict['database']), encoding="utf8")


    df_final.to_sql('Trix', con = engine, if_exists = 'append',\
                index = False, chunksize = 1000)